# QPFAS: Dask Workflow
Here we illustrate the Dask workflow by running a set of experiments defined by the yaml file `H2_exp_input.yml` which is in the `experiment_inputs` directory. The results will be saved in the file `H2_exp_output.json`.

This set of experiemnts is for H2 with the VQE run to determine the ground-state energy.


Before we begin, let's view the input yml file

In [ ]:
!cat experiment_inputs/H2_exp_input.yml

This will result in 6 VQE runs since we have 2 qubit transformations and 3 optimizers

In [ ]:
import os
import qpfas
from dask.distributed import get_task_stream  # necessary for the timings

## Setup Client

In [ ]:
client = qpfas.workflow.DaskDAG.create_client(dask_scheduler_uri="tcp://dask-scheduler:8786")

## Define experiments from YAML, generate DAGs from YAML, execute using Dask

In [ ]:
YAML_FILE = os.path.join(os.getcwd(), 'experiment_inputs','H2_exp_input.yml')
graph, keys_to_gather, exp_params, dag_nodes = qpfas.workflow.EfficientDAG.load_from_yaml(YAML_FILE)

print("Number of Experiments:", len(exp_params))

print("\nDAG Nodes:")
for i in dag_nodes:
    print(f"{i}: {dag_nodes[i]}")
    
print("\nKeys Gathered:")
print(keys_to_gather)

## View DAG

In [ ]:
qpfas.workflow.DaskDAG.visualize_dag(graph)

View single DAG for all experiments

## Execute DAG and get results

In [ ]:
metadata = qpfas.utils.get_all_metadata()

global_start_timestamp = qpfas.utils.get_datetime_now()
with get_task_stream() as ts:  # necessary for timing logs
    results = qpfas.workflow.DaskDAG.get(client, graph, keys_to_gather)
global_end_timestamp = qpfas.utils.get_datetime_now()

metadata['global_start_timestamp'] = global_start_timestamp
metadata['global_end_timestamp'] = global_end_timestamp

## Get timing logs

In [ ]:
results = qpfas.workflow.EfficientDAG.get_timing_logs(ts, exp_params, results, metadata)

## Combine dictionaries (params & results) and save to json

In [ ]:
qpfas.workflow.EfficientDAG.write_to_file("H2_exp_output", exp_params, results)

In [ ]:
!head -100 H2_exp_output.json